In [2]:
import os
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/gdrive/projects/programming/automation")

from time import sleep

import sistemas
import sei
import functions
import page
from page import *
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

% reload_ext autoreload
% autoreload 2

In [37]:
usr = "rsilva"
pwd = "Savorthemom3nts"

caps = DesiredCapabilities.INTERNETEXPLORER

caps['version'] = '11.0'
caps['requireWindowsFocus'] = True
caps['ignoreProtectedModeSettings'] = True

driver = webdriver.Ie(capabilities=caps)

In [38]:
scpx = sistemas.Scpx(driver)

In [49]:
cpf = '718.519.925-53'

In [50]:
cpf = cpf.replace(".","").replace('-',"").replace("/", "")

In [51]:
scpx.consulta(cpf)

In [15]:
scpx.servico_incluir(cpf)

In [16]:
scpx.incluir_estacao(cpf, 'Móvel', 'PX2T1988')

In [20]:
scpx.consulta(cpf)

In [17]:
scpx.movimento_aprovar(cpf, 'B')

In [18]:
scpx.licenciar_estacao(cpf, ppdess=True)

In [19]:
scpx.licenciar_estacao(cpf, ppdess=False)

In [5]:
os.listdir("files/csv")

['53504.0052872018-97.csv',
 '53504.0054462018-53.csv',
 '53504.0054632018-91.csv',
 '53504.0054672018-79.csv',
 '53504.0056342018-81.csv',
 '53504.005634_2018-81.csv',
 '53504.005639_2018-12.csv',
 '53504.0056652018-32.csv',
 '53504.0056672018-21.csv',
 'devolvidos.csv',
 'devolvidos.xlsx',
 'registros_cadin.csv',
 'registros_scpx_rf.csv']

In [45]:
import pandas as pd

registros = pd.read_csv("files/Expedidos.csv", dtype=str)

In [46]:
registros.head()

,Cpf,NOME,FISTEL,Unnamed: 3,Validade,End_Sede,End_Correspond.,Fone,Logradouro,Número,Complemento,Bairro,Cidade,Cep,Endereço_Completo,Processo
0,28086615847,ALEXANDRE RIBEIRO COUTINHO,80106084674,"80106084674,",17/05/2017,"RUA PROGRESSO, 21, casa 2\n- JARDIM SAO PAULO\...",RUA PITANGUEIRA 157 JD BARONESA \n CEP: 1310...,02033342,RUA FRANCISCO ANTONIO MIRANDA,88,NaN,EMERLINO MATARAZZO,SAO PAULO/SP,03809-130,RUA FRANCISCO ANTONIO MIRANDA 88 EMERLINO MAT...,53504.005890/2018-79
1,27654632800,ANTONIO MOTA DE SANTANA,80106059050,"80106059050,",18/05/2017,"RUA ADELINO MONAR, 170\n- SANTO ANTONIO\nCEP: ...",RUA ALFREDO FAVERO 262 JD PEDRO OMETTO \n CEP...,67835372,RUA ALBANO LAZIALE,33,NaN,JD. ARISE,SAO PAULO/SP,03574-040,RUA ALBANO LAZIALE 33 JD. ARISE \n CEP: 03574...,53504.005890/2018-79
2,97628239820,CARLOS BARBOSA DOS SANTOS,80105984450,"80105984450,",17/05/2017,"RUA JOSE NORBERTO PINTO, 317, casa\n- VILA ANA...",RESIDENCIAL RUA MAJOR JOSE INACIO 2553 CENTRO...,81677191,RUA MANOEL DOS SANTOS,321,NaN,PONTILHAO,CRUZEIRO/SP,12704-200,RUA MANOEL DOS SANTOS 321 PONTILHAO \n CEP: 1...,53504.005890/2018-79
3,09617513803,CELSO PEREIRA,80106050770,"80106050770,",17/05/2017,"RUA LUIZ MELGES, 25\n- CENTRO\nCEP: 17580000\n...",RUA SAO FRANCISCO 207 FDOS CENTRO \n CEP: 110...,00000000,RUA LUIZ MELGES,25,NaN,CENTRO,POMPEIA/SP,17580-000,RUA LUIZ MELGES 25 CENTRO \n CEP: 17580-000 P...,53504.005890/2018-79
4,27003217821,FABIO RIBEIRO VAZ LOBO FILHO,80106068121,"80106068121,",17/05/2017,"Rua Natal, 167, perto celsinho auto peças\n- V...",RUA MADALENA 83 CASA JD MARGARIDA \n CEP: 087...,92156103,RUA COSTA RICA,58,APARTAMENTO 2,JARDIM LAVINIA,MOCOCA/SP,13736-350,RUA COSTA RICA 58 APARTAMENTO 2 JARDIM LAVINIA...,53504.005890/2018-79


In [23]:
def extrai_pares_tabulação(source):
    trs = source.find_all('tr')
    dados = {}
    i = 1
    for tr in trs:        
        td = tr.find_all('td', string=True)
        label = tr.find_all('label', string=True)
        
        i = 1
        for field, result in zip(td, label):
            field, result = field.text[:-1], result.text
            if field in dados:
                field = field + "_" + str(i + 1)
            dados[field] = result 
       
    return dados

In [48]:
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup

for f in registros[registros["Processo"] == '53504.005977/2018-46'].iterrows():
    fistel = f[1].FISTEL
    while len(fistel) < 11:
        fistel = '0' + fistel
    scpx.consulta(fistel,tipo_id='id_fistel')
    source = soup(scpx.driver.page_source, "lxml")
    dados = extrai_pares_tabulação(source)
    
    scpx.wait_for_element_to_click((By.ID, "botaoFlatServiço")).click()
    val_rf = source.find(id='labelDataValidadeRadioFrequencia')
    
    #source = soup(scpx.driver.page_source, "lxml")
    #dados = {**dados, **extrai_pares_tabulação(source)}
                                               
    proximo = scpx.wait_for_element_to_click((By.ID, "botaoFlatEstação"))
    proximo.click()
    
    #source = soup(scpx.driver.page_source, "lxml")
    #dados = {**dados, **extrai_pares_tabulação(source)}
    
    print("Nome: {}".format(f[1]['NOME']))
    print("Validade de Radiofrequência: {}".format(val_rf.text))
    print("Data de Validade Licença: {}\n".format(dados['Data Validade Licença']))
    
    dados = {}

Nome: ANTONIO DE JESUS PINHEIRO
Validade de Radiofrequência: 19/06/2017
Data de Validade Licença: 19/06/2017

Nome: CRISTIANO SOARES DA SILVA
Validade de Radiofrequência: 16/06/2017
Data de Validade Licença: 16/06/2017

Nome: DIEGO HENRIQUE CORTEZ
Validade de Radiofrequência: 19/06/2017
Data de Validade Licença: 19/06/2017

Nome: EDSON NOGUEIRA GALEGO
Validade de Radiofrequência: 15/06/2017
Data de Validade Licença: 15/06/2017

Nome: JOAO PAULO DA SILVA NETO
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: JORGE LUIZ DE FREITAS
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: JOSE HENRIQUE DE PAULA
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: JULIO CARLOS VAZQUEZ
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: JURAFA PEDROSO
Validade de Radiofrequência: 13/06/2017
Data de Validade Licença: 13/06/2017

Nome: LUIS ROBERTO PIRES
Validade de Radio

In [16]:
for k,v in dados.items():
    print("{} : {}".format(k,v))

Tipo Estação : Móvel
Data Nascimento : 23/04/1960
Nacionalidade : brasileira
Usuário Alteração_2 : ANATEL\04233668660 (salatiel)
Usuário que Emitiu Licença : ANATEL\04233668660 (salatiel)
Data Alteração_2 : 19/12/2007 09:28:09
Logradouro : RUA IRIRI-MIRIM
Data Alteração : 22/01/2008 12:31:55
Bairro : JARDIM SANTA TEREZINHA (ZONA LESTE)
Usuário Alteração : ANATEL\52973018820 (soske)
Número Fistel : 80106420887
Situação_2 : L - Licenciado
Boleto PPDUR : 801064208870003
Usuário Inclusão : ANATEL\04233668660 (salatiel)
Usuário que Licenciou : ANATEL\04233668660 (salatiel)
Estações Excluídas : 0
Tipo Usuário : 3 - Integral
Usuário Inclusão_2 : ANATEL\04233668660 (salatiel)
Boleto TFI : 801064208870002
Usuário de Alteração : ANATEL\04233668660 (salatiel)
Complemento : CASA 01
Boleto PPDESS : 801064208870001
Nome Indicativo : PX2Y7176-001
Data Inclusão_2 : 21/11/2007 15:31:04
Data Emissão Licença : 19/12/2007 09:28:09
Sexo : Masculino
Data Inclusão : 21/11/2007 15:30:25
Cep : 03572180
Validad

In [20]:
dados["Validade RNE"]

KeyError: 'Validade RNE'

In [43]:
database = pd.DataFrame()

In [44]:
for v in banco_de_dados.values():
    db = pd.DataFrame([v])
    
    database = database.append(db)

In [45]:
database.head()

,Boleto 2ªVia,Boleto PPDESS,Boleto TFI,CNPJ ou CPF,CNPJ/CPF,Celular,Cep,Data Alteração,Data Emissão Licença,Data Exclusão,...,Situação,Status,Status da Licença,Tipo Usuário,Tipo de Comparação,UF,Validade RNE,Validade Radiofreqüência,Vencidas,Vincendas
0,,801059155640030,,,21451007841,NaN,RUA GABRIEL COURY ATHIE,ANATEL\52967522600 (deleicfaria),,ANATEL\52967522600 (deleicfaria),...,L - Licenciado,L - Licenciado,Licenciamento pendente por PRORROGAÇÃO,3 - Integral,,,,53504.014299/2007-50 (SICAP),,
0,,801060843210003,,,12942851813,NaN,ESTRADA BELVEDERE,ANATEL\13049323892 (luizmoura),ANATEL\13049323892 (luizmoura),,...,L - Licenciado,L - Licenciado,,3 - Integral,,,,53504.007461/2007-83 (SICAP),,
0,,801060528020003,,,13567801805,NaN,RUA GUATEMALA,ANATEL\13049323892 (luizmoura),ANATEL\13049323892 (luizmoura),,...,L - Licenciado,L - Licenciado,,3 - Integral,,,,53504.004642/2007-58 (SICAP),,
0,,801059442380003,,,33650494876,NaN,RUA JOAQUIM NABUCO,ANATEL\25091240802 (rogeriofl),ANATEL\25091240802 (rogeriofl),,...,L - Licenciado,L - Licenciado,,3 - Integral,,,,53504.024685/2006-79 (SICAP),,
0,,801060541710003,,,00171900880,NaN,RUA AMPARO,ANATEL\31888916877 (rsilva),ANATEL\13049323892 (luizmoura),,...,L - Licenciado,L - Licenciado,Licenciamento pendente por ALTERAÇÃO de CNPJ/C...,3 - Integral,,,,53504.004351/2007-60 (SICAP),,


In [5]:
scpx.incluir_serviço(CPF)

In [6]:
scpx.aprovar_movimento(CPF, "B")

In [8]:
scpx.licenciar_estacao(CPF)

In [9]:
scpx.consulta(CPF)

In [17]:
datas = ["23052018", "24052018"]
horas = ["1300", "1400"]

horarios = [(dia,hora) for dia in datas for hora in horas][2:]

print(horarios)

sistemas.abrir_agenda_prova(browser, horarios)

[('24052018', '1300'), ('24052018', '1400')]
